# How to backtest 2,000,000 simulations for the best exits

A backtest is a simulation of market dynamics that are used to test how trading strategies might perform.

The problem is that the markets are noisy and difficult to model.

So when running backtests, optimized models often fit to noise, instead of market inefficiencies. When applying the optimized parameters to previously unseen data, the model falls apart (along with your portfolio).

All parameters of a strategy affect the result but only a few determine entry and exit dependent on the market price. These are the parameters that should be optimized.

For example, is a 1% trailing stop better than a $1 stop loss?

We’ll use the cutting-edge backtesting framework vectorbt to optimize the entry and exit type for a momentum strategy.

In [1]:
import pytz
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import vectorbt as vbt

ModuleNotFoundError: No module named 'vectorbt'